In [5]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("ForEachPartition") \
    .getOrCreate()

In [6]:
from pyspark.sql import types as T

schema = T.StructType([
    T.StructField("created_at", T.TimestampType(), True),
    T.StructField("sentiment_level", T.StringType(), False)
])

In [7]:
df = spark.readStream.schema(schema).json('./data/')
df

DataFrame[created_at: timestamp, sentiment_level: string]

In [8]:
import socket
host = "localhost"
port = 6666

s = socket.socket()
def foreach_batch_function(df, epoch_id):
    for data in df.toJSON().collect():
        s = socket.socket()
        s.connect((host, port))
        data = str(data)
        s.send(data.encode("utf8"))
        s.close()
        
launch = df.writeStream.foreachBatch(foreach_batch_function).start()

In [9]:
launch.stop()